In [1]:
import netCDF4 as nc
from os import path

def add_diff(ncinfname1, ncinfname2, ncoutfname, toexclude):
    with nc.Dataset(ncinfname1) as src1, \
         nc.Dataset(ncinfname2) as src2, \
         nc.Dataset(ncoutfname, "w") as dst:
        # copy global attributes all at once via dictionary
        dst.setncatts(src1.__dict__)
        # copy dimensions
        for name, dimension in src1.dimensions.items():
            dst.createDimension(
                name, (len(dimension) if not dimension.isunlimited() else None))
        # copy all file data except for the excluded
        for name, variable in src1.variables.items():
            if name not in toexclude:
                x = dst.createVariable(name, variable.datatype, variable.dimensions)
                # copy variable attributes all at once via dictionary
                dst[name].setncatts(src1[name].__dict__)
                dst[name][:] = src1[name][:]

        # add diff
        dst_name = 'zeta_max_diff'
        x = dst.createVariable(dst_name, variable.datatype, variable.dimensions)
        attr = src1['zeta_max'].__dict__
        attr['long_name'] = 'maximum water surface elevation difference'
        attr['standard_name'] = 'zeta_max_diff'
        dst[dst_name].setncatts(attr)
        zeta_max1 = src1['zeta_max'][:]
        zeta_max2 = src2['zeta_max'][:]
        zeta_max_diff = zeta_max1 - zeta_max2
        dst[dst_name][:] = zeta_max_diff


basedir = "/projects/storm_surge/COMT/sbunya/runs/EmbedVEW1DtoLargeModel/NC/Florence"

ncfins1 = ["SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5/maxele.63.nc",
          "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5/maxele.63.nc",
          "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5/maxele.63.nc"]
ncfins2 = ["SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_noflows_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_swan2noswan_ERA52OWI2ERA5/maxele.63.nc",
          "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_nomet/maxele.63.nc",
          "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_noflows_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_swan2noswan_ERA52OWI2ERA5/maxele_max_costal_and_runoff.63.nc"]
ncfous = ["SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5/maxele_diff_compound-coastal.63.nc",
          "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5/maxele_diff_compound-runoff.63.nc",
          "SPST_27.05ca1_res80m_raised_slp0020005_nlim5m_usgs_nwmothers_dt2_velmin05_ccap_nrv15n0304tar_openwater02_slim001_tau005_Smag01_cn40m_swan2noswan_ERA52OWI2ERA5/maxele_diff_compound-max_coastal_and_runoff.63.nc"]

# toexclude = ['depth', 'zeta_max', 'time_of_zeta_max']
toexclude = []

for i in range(len(ncfins1)):
    print(i)
    ncfname_in1 = path.join(basedir, ncfins1[i])
    ncfname_in2 = path.join(basedir, ncfins2[i])
    ncfname_ou = path.join(basedir, ncfous[i])
    add_diff(ncfname_in1, ncfname_in2, ncfname_ou, toexclude)



0
1
2
